In [2]:
import pandas as pd
import gc # Garbage collector

from utils.feature_scope import get_feature_scope
from utils.transformation_functions import *

# PNS 2013 Cleaning

In [3]:
df = pd.read_parquet("..\..\data\staged\PNS_2013.parquet")

print(f"Count of df: {len(df)}")
df.head()

Count of df: 222385


,V0001,V0024,UPA_PNS,V0006_PNS,UPA,V0015,V0020,V0022,V0026,V0031,...,V00293,VDC001,VDC002,VDD004,VDD004A,VDE001,VDE002,VDF002,VDF003,VDDATA
0,11,1110011,1100002,0001,110021877,01,2013,04,1,1,...,11112,04,02,2,2,1,1,00000840,00000210,20200715
1,11,1110011,1100002,0001,110021877,01,2013,04,1,1,...,,04,02,5,5,2,,00000840,00000210,20200715
2,11,1110011,1100002,0001,110021877,01,2013,04,1,1,...,,04,02,,.,,,00000840,00000210,20200715
3,11,1110011,1100002,0001,110021877,01,2013,04,1,1,...,,04,02,4,4,2,,00000840,00000210,20200715
4,11,1110011,1100002,0002,110021877,01,2013,02,1,1,...,11113,02,02,2,2,1,1,00001000,00000500,20200715


## Rows to filter

In [4]:
# Only Females
only_females = df['C006'] == "2"

# Apply filters
df_filtered = df[only_females]
df_filtered = df_filtered[df_filtered['C008'].str.strip() != "."]
df_filtered = df_filtered[df_filtered['C008'].str.strip() != "" ]

print(f"Count of df_filtered: {len(df_filtered)}")
df_filtered.head()

Count of df_filtered: 106310


,V0001,V0024,UPA_PNS,V0006_PNS,UPA,V0015,V0020,V0022,V0026,V0031,...,V00293,VDC001,VDC002,VDD004,VDD004A,VDE001,VDE002,VDF002,VDF003,VDDATA
1,11,1110011,1100002,0001,110021877,01,2013,04,1,1,...,,04,02,5,5,2,,00000840,00000210,20200715
2,11,1110011,1100002,0001,110021877,01,2013,04,1,1,...,,04,02,,.,,,00000840,00000210,20200715
3,11,1110011,1100002,0001,110021877,01,2013,04,1,1,...,,04,02,4,4,2,,00000840,00000210,20200715
5,11,1110011,1100002,0002,110021877,01,2013,02,1,1,...,,02,02,2,2,2,,00001000,00000500,20200715
7,11,1110011,1100002,0004,110021877,01,2013,03,1,1,...,11122,03,02,5,5,2,,00004178,00001393,20200715


## Columns to filter

In [5]:
columns_to_select = []

for modulo in get_feature_scope():
    for column in modulo:
        for key in column.keys():
            if key in df_filtered.columns:
                columns_to_select.append(key)    

df_filtered = df_filtered[columns_to_select]

## Columns to clean

In [6]:
# Replace any column that has only spaces and "." by "<None>"
df_cleaned = df_filtered.apply(lambda x: x.map(lambda y: "<None>" if isinstance(y, str) and (y.strip() == '.' or y.strip() == '') else y))
df_cleaned

,V0020,C008,C009,R010,R011,R012,R025,R028,R031,R034,...,Q092,Q111,Q120,Q124,Q125,Q128,C008,C009,P027,P050
1,2013,037,1,<None>,<None>,<None>,<None>,<None>,<None>,<None>,...,<None>,<None>,<None>,<None>,<None>,<None>,037,1,<None>,<None>
2,2013,000,4,<None>,<None>,<None>,<None>,<None>,<None>,<None>,...,<None>,<None>,<None>,<None>,<None>,<None>,000,4,<None>,<None>
3,2013,016,1,<None>,<None>,<None>,<None>,<None>,<None>,<None>,...,<None>,<None>,<None>,<None>,<None>,<None>,016,1,<None>,<None>
5,2013,038,1,<None>,<None>,<None>,<None>,<None>,<None>,<None>,...,<None>,<None>,<None>,<None>,<None>,<None>,038,1,<None>,<None>
7,2013,032,2,2,<None>,<None>,12,<None>,1,2,...,2,<None>,2,2,<None>,2,032,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222378,2013,054,1,<None>,<None>,<None>,<None>,<None>,<None>,<None>,...,<None>,<None>,<None>,<None>,<None>,<None>,054,1,<None>,<None>
222380,2013,016,1,<None>,<None>,<None>,<None>,<None>,<None>,<None>,...,<None>,<None>,<None>,<None>,<None>,<None>,016,1,<None>,<None>
222381,2013,031,4,2,<None>,<None>,14,<None>,1,2,...,2,<None>,2,2,<None>,2,031,4,1,3
222383,2013,014,4,<None>,<None>,<None>,<None>,<None>,<None>,<None>,...,<None>,<None>,<None>,<None>,<None>,<None>,014,4,<None>,<None>


## Columns to transform

In [7]:
df_transformed = df_cleaned.copy()

# Dynamically call the transformation function based on the column name
for column in df_transformed.columns:

    transform_function_name = f"transform_{column}"

    if transform_function_name in globals():
        transform_function = globals()[transform_function_name]
        df_transformed = transform_function(df_transformed)    

df_transformed

,V0020,C008,C009,R010,R011,R012,R025,R028,R031,R034,...,Q092,Q111,Q120,Q124,Q125,Q128,C008,C009,P027,P050
1,2013,37,1,<None>,<None>,0,0,<None>,<None>,<None>,...,<None>,99,<None>,<None>,99,<None>,37,1,<None>,<None>
2,2013,0,4,<None>,<None>,0,0,<None>,<None>,<None>,...,<None>,99,<None>,<None>,99,<None>,0,4,<None>,<None>
3,2013,16,1,<None>,<None>,0,0,<None>,<None>,<None>,...,<None>,99,<None>,<None>,99,<None>,16,1,<None>,<None>
5,2013,38,1,<None>,<None>,0,0,<None>,<None>,<None>,...,<None>,99,<None>,<None>,99,<None>,38,1,<None>,<None>
7,2013,32,2,2,<None>,0,12,<None>,1,2,...,2,99,2,2,99,2,32,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222378,2013,54,1,<None>,<None>,0,0,<None>,<None>,<None>,...,<None>,99,<None>,<None>,99,<None>,54,1,<None>,<None>
222380,2013,16,1,<None>,<None>,0,0,<None>,<None>,<None>,...,<None>,99,<None>,<None>,99,<None>,16,1,<None>,<None>
222381,2013,31,4,2,<None>,0,14,<None>,1,2,...,2,99,2,2,99,2,31,4,1,3
222383,2013,14,4,<None>,<None>,0,0,<None>,<None>,<None>,...,<None>,99,<None>,<None>,99,<None>,14,4,<None>,<None>


In [8]:
# Check for duplicated columns
duplicated_cols = df_transformed.columns[df_transformed.columns.duplicated()].tolist()
print(f"Number of duplicated columns: {len(duplicated_cols)}")
if len(duplicated_cols) > 0:
    print("Duplicated columns:")
    print(duplicated_cols)


Number of duplicated columns: 4
Duplicated columns:
['C008', 'C009', 'P027', 'P050']


## Write to stage path

In [7]:
df_transformed.to_parquet("..\..\data\staged\PNS_2013_transformed.parquet")

In [ ]:
# Delete the dfs that are no longer needed
del df
del df_filtered
del df_cleaned
del df_transformed

# Run garbage collection to free up memory
gc.collect()